In [ ]:
## Homework 1
# pubmed_cleaned.zip is a file containing a metadatas sample from pubmed articles. Your goal is to convert the json to a mongo DB and answer the following questions USING mongodb querys:

In [15]:
import json
import pymongo
from bson import ObjectId

In [19]:
client = pymongo.MongoClient('localhost', 27017)
db = client["hw"]
collection = db["pubmed"]
with open('/Users/zac/Downloads/pubmed_cleaned.json') as file:
    file_data = json.load(file)

In [20]:
for document in file_data:
    if '_id' in document and '$oid' in document['_id']:
        document['_id'] = ObjectId(document['_id']['$oid'])
collection.insert_many(file_data)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 646004f76d879d04301864df, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [48]:
doc = collection.find_one()
print(doc)

{'_id': ObjectId('60c715b4c4b7559e04fd9d24'), 'pmid': 30006494, 'title': 'Proton pump inhibitors and gastric cancer: association is not causation.', 'ISSN': '1468-3288', 'abstract': None, 'meshwords': 'Anti-Ulcer Agents \nHelicobacter pylori \nHumans \nProton Pump Inhibitors \nStomach Neoplasms ', 'meshsubwords': '', 'authors': "name ml Moayyedi P, affil str Division of Gastroenterology and Farncombe Family Digestive Health Research Institute, McMaster University, Hamilton, Ontario, Canada.\nname ml Veldhuyzen van Zanten SJO, affil str Division of Gastroenterology, University of Alberta, Edmonton, Alberta, Canada.\nname ml Hookey L, affil str Division of Gastroenterology, Queen\\'s University, Kingston, Ontario, Canada.\nname ml Armstrong D, affil str Division of Gastroenterology and Farncombe Family Digestive Health Research Institute, McMaster University, Hamilton, Ontario, Canada.\nname ml Jones N, affil str Division of Gastroenterology, Hepatology, and Nutrition, The Hospital for S

In [12]:
# 1. Create an index, explain your choice of key.

collection.create_index([ ("id",1) ])

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 646002776d879d04301864dc, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [54]:
# 2. Delete every paper that was published prior 2019

query={"date":{'$regex':'^year 201[^9]','$options':'i'}}
db.hw.count_documents(query)
db.hw.delete_many(query)

In [63]:
# 3. How many paper have a single author ? Two authors ?

num_single_author_documents = 0
num_two_authors_documents = 0

# Compter le nombre de documents avec un seul auteur et deux auteurs
for document in collection.find():
    authors = document['authors'].split('\n')
    num_authors = len(authors)
    if num_authors == 1:
        num_single_author_documents += 1
    elif num_authors == 2:
        num_two_authors_documents += 1

# Afficher les résultats
print("Nombre de documents avec un seul auteur :", num_single_author_documents)
print("Nombre de documents avec deux auteurs :", num_two_authors_documents)

Nombre de documents avec un seul auteur : 5480
Nombre de documents avec deux auteurs : 10153


In [64]:
# 4. The last paper inserted in the db

last = collection.find().sort("id", -1).limit(1)
print(list(last))

[{'_id': ObjectId('60c753bac4b7559e040fe2b3'), 'pmid': 31226374, 'title': 'Identification of a novel UDP-glycosyltransferase gene from Rhodiola rosea and its expression during biotransformation of upstream precursors in callus culture.', 'ISSN': '1879-0003', 'abstract': ' "Roseroot (Rhodiola rosea L.) is a medicinal plant with adaptogenic properties and several pharmaceutically important metabolites. In this study, a full length cDNA encoding a UDPG gene of roseroot was identified, cloned and characterized. Its ORF (1425bp) was transferred into E. coli, where the expression of the recombinant enzyme was confirmed. To monitor the enzyme activity, 3 precursors (tyramine, 4-hydroxyphenylpyruvate & tyrosol) of salidroside biosynthesis pathway were added to roseroot callus cultures and samples were harvested after 1, 6, 12, 24, 48 & 96h. Along with the controls (without precursor feeding), each sample was subjected to HPLC and qRT-PCR for phytochemical and relative UDP-glycosyltransferase g

In [65]:
# 5. Find articles with null meshwords.

docs = db.hw.find({"meshwords":None})
for doc in docs:
    print(doc['title'])

In [67]:
# 6. Choose a keyword you are interested in (machine learning, computer vision,...). 

#Find the number of articles with the choosen keyword in their meshwords, abstract or title.

query = {"$or": [ { "meshwords": { "$regex": 'computer vision' }},{"abstract": { "$regex": 'computer vision' }},{"title": { "$regex": 'computer vision' }}]}
docs = collection.count_documents( query )
print (docs)

35


In [68]:
# 7. What's the number of articles that have at least one affiliation AND meshwords.

query = {"$and":[{"meshwords":{'$exists': 1}},{"authors":{ "$regex": 'affil' }}]}
docs = collection.count_documents( query )
print (docs)

99413


In [69]:
# 8. How many articles have a publishing date after 2020 ?

twothousandtwenty_article = collection.count_documents({'date_accepted' :{'$regex': ".*2021.*"}})
print("Numbers of articles published after 2020 is : ", twothousandtwenty_article)

Numbers of articles published after 2020 is :  0


In [71]:
# 9. Find articles where there's atleast one affiliation from a choosen country (you decide which one).

query = {"authors":{ "$regex": 'France' }}
docs = collection.count_documents( query )
print (docs)

4640


In [72]:
# 10. Check for any duplicates. (hint: look at the doi or the pmid)

len("field")!= len(set ("field"))

False

In [73]:
# 11. Remove every articles where the abstract starts with an "R".

collection.delete_many({'abstract': {'$regex': "\"R.*"}})

In [78]:
# 12. Return the list of papers (pmid) where there's atleast one affiliation per author

papers = collection.find({"authors": {"$exists": True}})
paper_list = [paper["pmid"] for paper in papers if len(paper.get("authors", [])) > 0]
print(paper_list)

[30608900, 30614081, 30614166, 30614610, 30614629, 30615595, 30615597, 30615818, 30615980, 30618194, 30618518, 30620089, 30620121, 30620713, 30621200, 30624690, 30624702, 30624841, 30625514, 30626152, 30626732, 30627924, 30629332, 30629810, 30629812, 30630063, 30630289, 30631788, 30631944, 30633307, 30635026, 30636019, 30636978, 30637770, 30637785, 30637966, 30638571, 30638935, 30639135, 30639237, 30639419, 30639457, 30639549, 30639777, 30639781, 30640037, 30641129, 30641131, 30641134, 30641145, 30641149, 30641153, 30641246, 30641387, 30641886, 30641906, 30642096, 30642391, 30642749, 30642780, 30642903, 30644080, 30644106, 30644172, 30644615, 30645024, 30645031, 30645189, 30645190, 30645676, 30645685, 30646603, 30646842, 30646862, 30647233, 30647234, 30647235, 30647236, 30647237, 30647238, 30647239, 30647240, 30647241, 30647242, 30647243, 30647244, 30647245, 30647246, 30647247, 30647248, 30647249, 30647250, 30647251, 30647252, 30647253, 30647254, 30647255, 30647256, 30647257, 30647258,

In [80]:
# 13. Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples

import numpy as np
art=[{"array":list(np.random.random(500))},
          {"array":list(np.random.random(500))},
          {"array":list(np.random.random(500))}]
collection.insert_many(art)
docs=collection.find()
docs